In [1]:
import pandas as pd
import requests

In [2]:
base_event_url = 'https://esports-api.lolesports.com/persisted/gw/getCompletedEvents?hl=en-GB&tournamentId={tournament_id}'
auth_token = '0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z'

In [3]:
tournament_df = pd.read_csv('../data/tournament_codes.csv')

In [4]:
tournament_df[tournament_df['league'] == 'lpl']

,league,id,slug,startDate,endDate,year,season
104,lpl,111561319409710508,lpl_spring_2024,2024-01-21,2024-04-21,2024,spring
105,lpl,110825936250664572,lpl_regional_finals_2023,2023-08-03,2023-08-10,2023,regional
106,lpl,110428848766564346,lpl_summer_2023,2023-05-28,2023-08-19,2023,summer
107,lpl,109669600527985422,lpl_spring_2023,2023-01-12,2023-04-16,2023,spring
108,lpl,108888310291632913,lpl_regional_finals_2022,2022-09-01,2022-09-05,2022,regional
109,lpl,108431300950695970,lpl_summer_2022,2022-06-09,2022-09-01,2022,summer
110,lpl,107417779630700437,lpl_spring_2022,2022-01-10,2022-05-01,2022,spring
111,lpl,106860829994219982,lpl_regional_finals_2021,2021-08-29,2021-09-06,2021,regional
112,lpl,106269484328946755,lpl_summer_2021,2021-06-05,2021-09-06,2021,summer
113,lpl,105516880821527383,lpl_spring_2021,2021-01-09,2021-04-18,2021,spring


In [ ]:
tournament_df= tournament_df[(tournament_df['league'].isin(['lec','lcs','lpl','lck'])) & (tournament_df['year'] == '2024' ) & (tournament_df['season'] != 'challengers' )]

In [ ]:
tournament_df

In [ ]:
def process_match_data(json_data):
    new_record = dict()
    new_record['startTime'] = json_data['startTime']
    new_record['blockName'] = json_data['blockName']
    new_record['league'] = json_data['league']['name']
    new_record['match_id'] = json_data['match']['id']
    new_record['type'] = json_data['match']['type']
    new_record['team_1_name'] = json_data['match']['teams'][0]['name']
    new_record['team_1_code'] = json_data['match']['teams'][0]['code']
    new_record['team_1_result'] = json_data['match']['teams'][0]['result']['gameWins']
    new_record['team_2_name'] = json_data['match']['teams'][1]['name']
    new_record['team_2_code'] = json_data['match']['teams'][1]['code']
    new_record['team_2_result'] = json_data['match']['teams'][1]['result']['gameWins']

    new_record['format'] = json_data['match']['strategy']['type'] +'_'+ str(json_data['match']['strategy']['count'])
    games_list = []
    for g in json_data['games']:
        games_list.append([g['id'],g['vods']])

    new_record['games'] = games_list

    return new_record

In [ ]:
temp_list = []
for league in tournament_df['id']:
    url = base_event_url.format(tournament_id = league)
    json_data = requests.get(url,headers={'x-api-key': auth_token}).json()
    if len(json_data['data']['schedule']['events']) == 0:
        continue
    
    for event in json_data['data']['schedule']['events']:
        new_record = process_match_data(event)
        temp_list.append(new_record)

In [ ]:

event_df = pd.DataFrame.from_dict(temp_list)

In [ ]:
event_df.to_csv('../data/event.csv',index=False)

In [ ]:
tournament_df.query("league == 'lck'")

In [ ]:
tournament_df.query("league == 'lpl'")